In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from collections import OrderedDict
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
import torchvision

import torchvision.models as models
model = models.resnet18(pretrained=True)

test_dir = '../images/testing'
train_dir = '../images/training'

train_files = []

for subfolder in os.listdir(train_dir):
    print(subfolder)
    if subfolder == '.DS_Store':
        continue
    for files in os.listdir(os.path.join(train_dir, subfolder)):
        train_files.append(os.path.join(subfolder, files))

test_files = []
for subfolder in os.listdir(test_dir):
    #print(subfolder)
    if subfolder == '.DS_Store':
        continue
    for files in os.listdir(os.path.join(test_dir, subfolder)):
        test_files.append(os.path.join(subfolder, files))

print(len(train_files))
print(len(test_files))

labels = ["glass", "paper", "metal", "plastic"]

class WasteDataset(Dataset):
    def __init__(self, file_list, dir, mode='train', transform = None):
        self.file_list = file_list
        self.dir = dir
        self.mode= mode
        self.transform = transform
        if self.mode == 'train':
            for i, label in enumerate(labels):
                if label in self.file_list[0]:
                    self.label = i
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.dir, self.file_list[idx]))
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.file_list[idx]
        
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ColorJitter(),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(128),
    transforms.ToTensor()
])

class_files = [0] * len(labels)
for i, label in enumerate(labels):
    class_files[i] = [fileName for fileName in train_files if label in fileName]

class_datasets = [0] * len(labels)
for i in range(len(labels)):
    class_datasets[i] = WasteDataset(class_files[i], train_dir,  transform=data_transform)

waste_data = ConcatDataset(class_datasets)
dataloader = DataLoader(waste_data, batch_size = 64, shuffle=True, num_workers=4)

samples, labels = iter(dataloader).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
# in tensor, image is (batch, width, height), so you have to transpose it to (width, height, batch) in numpy to show it.
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))
#plt.waitforbuttonpress()

PermissionError: [Errno 13] Permission denied: '/Users/charlieliu/.cache/torch/checkpoints/resnet18-5c106cde.pth'